In [ ]:
import pandas as pd

In [ ]:
year = 2020
month = 12

monthly_data = pd.read_parquet(f'{year}/{year}-{month:02d}.parquet')

### Fontos oszlopok kiválasztása

Az eredeti adathalmaz 19 oszlopa közül kiválasztottuk a számunkra érdekesebbeket.

In [ ]:
monthly_data = monthly_data[
    ['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 'trip_distance', 'PULocationID',
     'DOLocationID', 'payment_type', 'fare_amount', 'extra', 'tip_amount', 'total_amount']
]

### Outlierek kiszűrése

Pár oszlopnál tapasztaltunk rendkívül magas/alacsony értékeket (például 2088-as dátum).

Az alábbi cella az 1-99-es percentillisig tartja meg az adatokat a `filer_columns`-ban található oszlopokban, ezzel kiszűrve az extrém értékeket.

A dátumoknál meghatároztuk, hogy az adott hónap időkeretében kell lennie a `tpep_pickup_datetime` és a `tpep_dropoff_datetime`-nak.

Az is fontos, hogy a `tpep_pickup_datetime` ne legyen előbb mint a `tpep_dropoff_datetime`.

In [ ]:
filter_columns = ['passenger_count', 'trip_distance', 'fare_amount', 'extra', 'tip_amount', 'total_amount']
percentile = .01

filter_string = ' & '.join(
    (
        *(f'({column}.quantile({percentile}) <= {column}) & ({column} <= {column}.quantile({1 - percentile}))' for column in filter_columns),
        f'("{year}, {month}, 1" <= tpep_pickup_datetime) & (tpep_pickup_datetime <= "{year}, {month}, 31")',
        f'("{year}, {month}, 1" <= tpep_dropoff_datetime) & (tpep_dropoff_datetime <= "{year}, {month}, 31")',
        'tpep_pickup_datetime <= tpep_dropoff_datetime'
    )
)

monthly_data = monthly_data.query(filter_string)